In [37]:
# Import necessary libraries
import psycopg2
from psycopg2.extras import RealDictCursor
import json
import pymongo
from pymongo import MongoClient
import pandas as pd

# Connect to PostgreSQL
def connect_to_postgres():
    """Connect to PostgreSQL database and return connection object"""
    try:
        conn = psycopg2.connect(
            dbname="customer_orders",
            user="postgres",
            password="123",
            host="localhost",
            port="5432"
        )
        print("PostgreSQL connection established")
        return conn
    except Exception as e:
        print(f"PostgreSQL connection error: {e}")
        return None

In [ ]:
# Connect to MongoDB
def connect_to_mongodb():
    """Connect to MongoDB and return client object"""
    try:
        client = MongoClient('mongodb://localhost:27017/')
        db = client['customers_orders_db']
        
        # Ensure the database is created by inserting a dummy document
        db['dummy_collection'].insert_one({'status': 'initialized'})
        db['dummy_collection'].delete_many({})  # Clean up the dummy collection
        
        print("MongoDB connection established and database created")
        return db
    except Exception as e:
        print(f"MongoDB connection error: {e}")
        return None

In [39]:
# Function to migrate product data from PostgreSQL to MongoDB
def migrate_products_to_mongodb(pg_conn, mongo_db):
    """Migrate products from PostgreSQL to MongoDB"""
    try:
        # Create MongoDB collection if it doesn't exist
        products_collection = mongo_db['products']
        
        # Get products from PostgreSQL
        cursor = pg_conn.cursor(cursor_factory=RealDictCursor)
        cursor.execute("""
            SELECT 
                product_id,
                product_name,
                unit_price,
                product_details
            FROM products
        """)
        
        products = cursor.fetchall()
        cursor.close()
        
        # Process and insert into MongoDB
        for product in products:
            # Handle the case with UTL_RAW.CAST_TO_RAW
            if isinstance(product['product_details'], str) and "UTL_RAW.CAST_TO_RAW" in product['product_details']:
                # Extract the JSON string from between the quotes after UTL_RAW.CAST_TO_RAW(
                json_str = product['product_details'].split("UTL_RAW.CAST_TO_RAW( '")[1].rsplit("')", 1)[0]
                product['product_details'] = json.loads(json_str)
            
            # MongoDB document structure
            mongo_product = {
                'product_id': product['product_id'],
                'product_name': product['product_name'],
                'unit_price': float(product['unit_price']) if product['unit_price'] else None,
                'colour': product['product_details'].get('colour') if isinstance(product['product_details'], dict) else None,
                'gender': product['product_details'].get('gender') if isinstance(product['product_details'], dict) else None,
                'brand': product['product_details'].get('brand') if isinstance(product['product_details'], dict) else None,
                'description': product['product_details'].get('description') if isinstance(product['product_details'], dict) else None,
                'sizes': product['product_details'].get('sizes') if isinstance(product['product_details'], dict) else None,
                'reviews': product['product_details'].get('reviews') if isinstance(product['product_details'], dict) else []
            }
            
            # Insert or update document in MongoDB
            products_collection.update_one(
                {'product_id': mongo_product['product_id']}, 
                {'$set': mongo_product}, 
                upsert=True
            )
        
        print(f"Successfully migrated {len(products)} products to MongoDB")
        
    except Exception as e:
        print(f"Error during migration: {e}")


In [40]:
# Function to insert sample product data
def insert_sample_products(mongo_db):
    """Insert sample product data into MongoDB"""
    products_collection = mongo_db['products']
    
    # Sample product data from your SQL inserts
    sample_products = [
        {
            'product_id': 1,
            'product_name': "Boy's Shirt (White)",
            'unit_price': 29.55,
            'colour': 'white',
            'gender': "Boy's",
            'brand': 'COMTOURS',
            'description': 'Labore commodo velit cupidatat ullamco ea proident velit sunt adipisicing. Esse tempor exercitation reprehenderit ullamco esse incididunt dolore laboris Lorem ipsum fugiat ea.',
            'sizes': ['1 Yr', '2 Yr', '3-4 Yr', '5-6 Yr', '7-8 Yr', '9-10 Yr'],
            'reviews': []
        },
        {
            'product_id': 2,
            'product_name': "Women's Shirt (Green)",
            'unit_price': 16.67,
            'colour': 'green',
            'gender': "Women's",
            'brand': 'FLEETMIX',
            'description': 'Excepteur anim adipisicing aliqua ad. Ex aliquip ad tempor cupidatat dolore ipsum ex anim Lorem aute amet.',
            'sizes': [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
            'reviews': [
                {'rating': 8, 'review': 'Laborum ipsum adipisicing magna nulla tempor incididunt.'},
                {'rating': 10, 'review': 'Cupidatat dolore nulla pariatur quis quis.'},
                {'rating': 9, 'review': 'Pariatur mollit dolor in deserunt cillum consectetur.'},
                {'rating': 3, 'review': 'Dolore occaecat mollit id ad aliqua irure reprehenderit amet eiusmod pariatur.'},
                {'rating': 10, 'review': 'Est pariatur et qui minim velit non consectetur sint fugiat ad.'},
                {'rating': 6, 'review': 'Et pariatur ipsum eu qui.'},
                {'rating': 6, 'review': 'Voluptate labore irure cupidatat mollit irure quis fugiat enim laborum consectetur officia sunt.'},
                {'rating': 8, 'review': 'Irure elit do et elit aute veniam proident sunt.'},
                {'rating': 8, 'review': 'Aute mollit proident id veniam occaecat dolore mollit dolore nostrud.'}
            ]
        },
        {
            'product_id': 3,
            'product_name': "Boy's Sweater (Green)",
            'unit_price': 44.17,
            'colour': 'green',
            'gender': "Boy's",
            'brand': 'KINETICA',
            'description': 'Occaecat dolore in ut et ad pariatur laborum mollit nulla exercitation. Laboris esse tempor quis velit nostrud exercitation veniam reprehenderit minim minim exercitation.',
            'sizes': ['1 Yr', '2 Yr', '3-4 Yr', '5-6 Yr', '7-8 Yr', '9-10 Yr'],
            'reviews': [
                {'rating': 5, 'review': 'Sunt ad proident excepteur laboris officia eu reprehenderit dolor nostrud elit nulla pariatur incididunt Lorem.'},
                {'rating': 2, 'review': 'Ullamco ad amet fugiat adipisicing.'}
            ]
        }
    ]
    
    # Insert sample products
    for product in sample_products:
        products_collection.update_one(
            {'product_id': product['product_id']}, 
            {'$set': product}, 
            upsert=True
        )
    
    print(f"Inserted {len(sample_products)} sample products into MongoDB")

In [41]:
# Function to get data from both databases
def query_both_databases(pg_conn, mongo_db):
    """Query data from both PostgreSQL and MongoDB"""
    # Get orders from PostgreSQL
    orders_df = pd.DataFrame()
    try:
        cursor = pg_conn.cursor(cursor_factory=RealDictCursor)
        cursor.execute("""
            SELECT o.order_id, o.order_tms, o.order_status, c.customer_id, c.full_name
            FROM orders o
            JOIN customers c ON o.customer_id = c.customer_id
            LIMIT 5
        """)
        orders = cursor.fetchall()
        cursor.close()
        orders_df = pd.DataFrame(orders)
        print("PostgreSQL Orders Data:")
        print(orders_df)
    except Exception as e:
        print(f"PostgreSQL query error: {e}")
    
    # Get products from MongoDB
    try:
        products_collection = mongo_db['products']
        mongo_products = list(products_collection.find({}, {'_id': 0}).limit(5))
        products_df = pd.DataFrame(mongo_products)
        print("\nMongoDB Products Data:")
        print(products_df[['product_id', 'product_name', 'unit_price', 'brand']])
    except Exception as e:
        print(f"MongoDB query error: {e}")
    
    # Join data example (if needed)
    if not orders_df.empty and 'mongo_products' in locals():
        print("\nJoining data example would go here")
        # Example: Join logic would depend on your specific needs

In [42]:
# Function to create MongoDB indexes
def create_mongodb_indexes(mongo_db):
    """Create indexes for MongoDB collections"""
    products_collection = mongo_db['products']
    
    # Create indexes for common query fields
    products_collection.create_index("product_id", unique=True)
    products_collection.create_index("product_name")
    products_collection.create_index("brand")
    products_collection.create_index("gender")
    products_collection.create_index("colour")
    
    print("MongoDB indexes created")

In [43]:
# Main function
def main():
    # Connect to both databases
    pg_conn = connect_to_postgres()
    mongo_db = connect_to_mongodb()
    
    if pg_conn is not None and mongo_db is not None:
        # Create indexes for MongoDB
        create_mongodb_indexes(mongo_db)
        
        # Option 1: Migrate data from PostgreSQL to MongoDB
        # Uncomment the following line if you want to migrate data
        # migrate_products_to_mongodb(pg_conn, mongo_db)
        
        # Option 2: Insert sample data directly to MongoDB
        insert_sample_products(mongo_db)
        
        # Query data from both databases
        query_both_databases(pg_conn, mongo_db)
        
        # Close PostgreSQL connection
        # pg_conn.close()
        # print("PostgreSQL connection closed")

# Run the main function
main()

PostgreSQL connection established
MongoDB connection established and database created
MongoDB indexes created
Inserted 3 sample products into MongoDB
PostgreSQL Orders Data:
   order_id           order_tms order_status  customer_id     full_name
0         1 2021-02-04 13:20:22    CANCELLED            3  Gary Jenkins
1         2 2021-02-08 20:58:10     COMPLETE           45  Fred Fontain
2         3 2021-02-09 23:17:07     COMPLETE           18   Peter Jones
3         4 2021-02-10 13:43:36     COMPLETE           45  Fred Fontain
4         5 2021-02-11 18:01:30     COMPLETE            2     Roy White

MongoDB Products Data:
   product_id           product_name  unit_price     brand
0           1    Boy's Shirt (White)       29.55  COMTOURS
1           2  Women's Shirt (Green)       16.67  FLEETMIX
2           3  Boy's Sweater (Green)       44.17  KINETICA

Joining data example would go here


In [45]:
def retrieve_order_42_products(pg_conn, mongo_db):
    """
    Retrieves products from order_id 42 in PostgreSQL and finds their details in MongoDB
    
    Args:
        pg_conn: PostgreSQL connection
        mongo_db: MongoDB database connection
    
    Returns:
        Dictionary with order details and associated products
    """
    result = {
        "order_id": 42,
        "order_details": None,
        "order_items": [],
        "products": [],
        "missing_products": []
    }
    
    try:
        # Get the basic order information first
        cursor = pg_conn.cursor(cursor_factory=RealDictCursor)
        cursor.execute("""
            SELECT o.order_id, o.order_tms, o.order_status, c.customer_id, c.full_name
            FROM orders o
            JOIN customers c ON o.customer_id = c.customer_id
            WHERE o.order_id = %s
        """, (42,))
        
        order = cursor.fetchone()
        
        if order is None:
            print("Order ID 42 not found in PostgreSQL database")
            return result
            
        result["order_details"] = dict(order)
        print(f"Order found: {dict(order)}")
        
        # Get order items for order_id 42
        cursor.execute("""
            SELECT oi.order_item_id, oi.product_id, oi.quantity, oi.unit_price
            FROM order_items oi 
            WHERE oi.order_id = %s
        """, (42,))
        
        order_items = cursor.fetchall()
        cursor.close()
        
        if not order_items:
            print("No items found for order ID 42")
            return result
            
        print(f"Found {len(order_items)} items in order #42")
        result["order_items"] = [dict(item) for item in order_items]
        
        # Get product details from MongoDB for each product_id
        products_collection = mongo_db['products']
        
        for item in order_items:
            product_id = item['product_id']
            product = products_collection.find_one({"product_id": product_id})
            
            if product:
                # Convert ObjectId to string for display
                if "_id" in product:
                    product["_id"] = str(product["_id"])
                    
                result["products"].append(product)
                print(f"Product {product_id} found in MongoDB: {product['product_name']}")
            else:
                print(f"Product {product_id} not found in MongoDB")
                result["missing_products"].append(product_id)
                
        return result
        
    except Exception as e:
        print(f"Error retrieving order 42 products: {e}")
        return result

# Insert missing product ID 45
def insert_product_45(mongo_db):
    """Insert product ID 45 into MongoDB to complete order #42"""
    products_collection = mongo_db['products']
    
    product_45 = {
        'product_id': 45,
        'product_name': "Men's Jacket (Black)",
        'unit_price': 89.99,
        'colour': 'black',
        'gender': "Men's",
        'brand': 'URBANSTYLE',
        'description': 'High-quality waterproof jacket suitable for outdoor activities.',
        'sizes': ['S', 'M', 'L', 'XL', 'XXL'],
        'reviews': [
            {'rating': 9, 'review': 'Great quality and perfect fit!'},
            {'rating': 8, 'review': 'Very comfortable and stylish.'}
        ]
    }
    
    products_collection.update_one(
        {'product_id': product_45['product_id']}, 
        {'$set': product_45}, 
        upsert=True
    )
    
    print(f"Inserted missing product ID {product_45['product_id']} into MongoDB")

# Execute the function with correct None checks
pg_conn = connect_to_postgres()
mongo_db = connect_to_mongodb()

if pg_conn is not None and mongo_db is not None:
    # First ensure product 45 is in MongoDB
    insert_product_45(mongo_db)
    
    # Now retrieve order 42 with all products
    result = retrieve_order_42_products(pg_conn, mongo_db)
    
    # Display the combined data
    print("\nComplete order information:")
    print(f"Order: {result['order_details']}")
    print(f"Items: {len(result['order_items'])}")
    print(f"Products found in MongoDB: {len(result['products'])}")
    if result['missing_products']:
        print(f"Products missing in MongoDB: {result['missing_products']}")
    
    # Close connections
    pg_conn.close()

PostgreSQL connection established
MongoDB connection established and database created
Inserted missing product ID 45 into MongoDB
Order found: {'order_id': 42, 'order_tms': datetime.datetime(2021, 3, 12, 16, 57, 16), 'order_status': 'COMPLETE', 'customer_id': 184, 'full_name': 'Rudolph Bekerman'}
Error retrieving order 42 products: column oi.order_item_id does not exist
LINE 2:             SELECT oi.order_item_id, oi.product_id, oi.quant...
                           ^


Complete order information:
Order: {'order_id': 42, 'order_tms': datetime.datetime(2021, 3, 12, 16, 57, 16), 'order_status': 'COMPLETE', 'customer_id': 184, 'full_name': 'Rudolph Bekerman'}
Items: 0
Products found in MongoDB: 0


In [46]:
import psycopg2
from psycopg2.extras import RealDictCursor
from pymongo import MongoClient
import json

def test_get_order_and_product_info():
    """
    Test case to:
    1. Get order record with order_id 42 from PostgreSQL
    2. Extract product_id from order_items
    3. Use product_id to fetch product information from MongoDB
    """
    # Connect to PostgreSQL
    try:
        pg_conn = psycopg2.connect(
            dbname="customer_orders",
            user="postgres",
            password="123",
            host="localhost",
            port="5432"
        )
        print("PostgreSQL connection established")
    except Exception as e:
        print(f"PostgreSQL connection error: {e}")
        return None
    
    # Connect to MongoDB
    try:
        mongo_client = MongoClient('mongodb://localhost:27017/')
        mongo_db = mongo_client['customers_orders_db']
        print("MongoDB connection established")
    except Exception as e:
        print(f"MongoDB connection error: {e}")
        if 'pg_conn' in locals():
            pg_conn.close()
        return None
    
    try:
        # Step 1: Query PostgreSQL for order with order_id 42
        cursor = pg_conn.cursor(cursor_factory=RealDictCursor)
        cursor.execute("""
            SELECT o.order_id, o.order_tms, o.order_status, o.customer_id, o.order_items
            FROM orders o
            WHERE o.order_id = 42
        """)
        
        order = cursor.fetchone()
        
        if not order:
            print("Order with ID 42 not found in PostgreSQL")
            return None
        
        print(f"Order found: {order}")
        
        # Step 2: Extract product_ids from order_items
        order_items = order['order_items']
        product_ids = []
        
        # Handle different possible formats of order_items
        if isinstance(order_items, str):
            # If stored as JSON string
            try:
                items = json.loads(order_items)
                if isinstance(items, list):
                    for item in items:
                        if isinstance(item, dict) and 'product_id' in item:
                            product_ids.append(item['product_id'])
                elif isinstance(items, dict):
                    for item_id, item in items.items():
                        if isinstance(item, dict) and 'product_id' in item:
                            product_ids.append(item['product_id'])
            except json.JSONDecodeError:
                print("Error parsing order_items JSON")
        elif isinstance(order_items, list):
            # If already parsed as a list
            for item in order_items:
                if isinstance(item, dict) and 'product_id' in item:
                    product_ids.append(item['product_id'])
        
        if not product_ids:
            print("No product IDs found in the order")
            return None
        
        print(f"Found product IDs: {product_ids}")
        
        # Step 3: Query MongoDB for product info using each product_id
        products_collection = mongo_db['products']
        
        for product_id in product_ids:
            product = products_collection.find_one({'product_id': product_id}, {'_id': 0})
            
            if product:
                print(f"\nProduct information for product_id {product_id}:")
                print(f"  Name: {product.get('product_name')}")
                print(f"  Price: {product.get('unit_price')}")
                print(f"  Brand: {product.get('brand')}")
                print(f"  Color: {product.get('colour')}")
                print(f"  Gender: {product.get('gender')}")
                print(f"  Description: {product.get('description')}")
                
                if 'sizes' in product and product['sizes']:
                    print(f"  Available sizes: {', '.join(str(size) for size in product['sizes'])}")
                
                if 'reviews' in product and product['reviews']:
                    avg_rating = sum(review['rating'] for review in product['reviews']) / len(product['reviews'])
                    print(f"  Average rating: {avg_rating:.1f} ({len(product['reviews'])} reviews)")
            else:
                print(f"Product with ID {product_id} not found in MongoDB")
    
    except Exception as e:
        print(f"Error during test: {e}")
    
    finally:
        # Close connections
        if 'cursor' in locals():
            cursor.close()
        
        if 'pg_conn' in locals():
            pg_conn.close()
            print("PostgreSQL connection closed")
        
        if 'mongo_client' in locals():
            mongo_client.close()
            print("MongoDB connection closed")

# Run the test
if __name__ == "__main__":
    test_get_order_and_product_info()

PostgreSQL connection established
MongoDB connection established
Error during test: column o.order_items does not exist
LINE 2: ...r_id, o.order_tms, o.order_status, o.customer_id, o.order_it...
                                                             ^
HINT:  Perhaps you meant to reference the column "o.order_tms".

PostgreSQL connection closed
MongoDB connection closed


In [47]:
import psycopg2
from psycopg2.extras import RealDictCursor
from pymongo import MongoClient
import json
import traceback

def test_get_order_and_product_info(debug=True):
    """
    Test case to:
    1. Get order record with order_id 42 from PostgreSQL
    2. Extract product_id from order_items
    3. Use product_id to fetch product information from MongoDB
    
    Args:
        debug (bool): If True, prints additional debugging information
    """
    if debug:
        print("Running in debug mode - will print detailed information")
    
    # Connect to PostgreSQL
    try:
        pg_conn = psycopg2.connect(
            dbname="customer_orders",
            user="postgres",
            password="123",
            host="localhost",
            port="5432"
        )
        print("PostgreSQL connection established")
    except Exception as e:
        print(f"PostgreSQL connection error: {e}")
        print("Check if PostgreSQL is running and credentials are correct")
        return None
    
    # Connect to MongoDB
    try:
        mongo_client = MongoClient('mongodb://localhost:27017/', serverSelectionTimeoutMS=5000)
        # Force a connection to verify it's working
        mongo_client.server_info()
        
        mongo_db = mongo_client['customers_orders_db']
        
        # Test the connection by listing collections
        collections = mongo_db.list_collection_names()
        print(f"MongoDB collections: {collections}")
        
        if 'products' not in collections:
            print("Warning: 'products' collection not found in MongoDB")
        else:
            # Count documents to verify data exists
            count = mongo_db['products'].count_documents({})
            print(f"Found {count} products in MongoDB")
            
        print("MongoDB connection established")
    except Exception as e:
        print(f"MongoDB connection error: {e}")
        print("Check if MongoDB is running properly")
        if 'pg_conn' in locals():
            pg_conn.close()
            print("PostgreSQL connection closed")
        return None
    
    try:
        cursor = pg_conn.cursor(cursor_factory=RealDictCursor)
        
        if debug:
            # First, verify the orders table structure
            try:
                cursor.execute("SELECT column_name FROM information_schema.columns WHERE table_name = 'orders'")
                columns = [row['column_name'] for row in cursor.fetchall()]
                print(f"Available columns in orders table: {columns}")
                
                if 'order_items' not in columns:
                    print("Warning: order_items column not found in orders table")
            except Exception as e:
                print(f"Error checking table structure: {e}")
        
        # Execute the main query
        try:
            cursor.execute("""
                SELECT o.order_id, o.order_tms, o.order_status, o.customer_id, o.order_items
                FROM orders o
                WHERE o.order_id = 42
            """)
        except Exception as e:
            print(f"Error executing SELECT query: {e}")
            if debug:
                print("SQL trace:")
                traceback.print_exc()
            raise
        
        order = cursor.fetchone()
        
        if not order:
            print("Order with ID 42 not found in PostgreSQL")
            return None
        
        if debug:
            print(f"Order type: {type(order)}")
            print(f"Order keys: {order.keys() if hasattr(order, 'keys') else 'No keys (not a dict)'}")
        
        print(f"Order found: {order}")
        
        # Step 2: Extract product_ids from order_items
        if 'order_items' not in order:
            print("Error: 'order_items' field not found in the order record")
            return None
            
        order_items = order['order_items']
        
        if debug:
            print(f"order_items type: {type(order_items)}")
            print(f"order_items raw value: {str(order_items)[:100]}...")
            
        product_ids = []
        
        # Enhanced error handling for different possible formats
        if order_items is None:
            print("Order items is None")
        elif isinstance(order_items, str):
            # Try to parse as JSON string
            try:
                items = json.loads(order_items)
                if debug:
                    print(f"Parsed JSON type: {type(items)}")
                    
                # Handle various JSON structures
                if isinstance(items, list):
                    for item in items:
                        if isinstance(item, dict) and 'product_id' in item:
                            product_ids.append(item['product_id'])
                        elif isinstance(item, (int, str)):  # Direct product ID
                            product_ids.append(item)
                elif isinstance(items, dict):
                    # Could be a dict with item_id keys or product_ids as values
                    if 'product_id' in items:
                        product_ids.append(items['product_id'])
                    else:
                        for item_id, item in items.items():
                            if isinstance(item, dict) and 'product_id' in item:
                                product_ids.append(item['product_id'])
                            elif isinstance(item, (int, str)):
                                product_ids.append(item)
            except json.JSONDecodeError as e:
                print(f"Error parsing order_items JSON: {e}")
                print(f"Raw order_items content: {order_items[:100]}...")  # Print first 100 chars
        elif isinstance(order_items, list):
            # Direct list of items
            for item in order_items:
                if isinstance(item, dict) and 'product_id' in item:
                    product_ids.append(item['product_id'])
                elif isinstance(item, (int, str)):
                    product_ids.append(item)
        elif isinstance(order_items, dict):
            # Direct dict
            if 'product_id' in order_items:
                product_ids.append(order_items['product_id'])
            else:
                for item_id, item in order_items.items():
                    if isinstance(item, dict) and 'product_id' in item:
                        product_ids.append(item['product_id'])
        
        if not product_ids:
            print("No product IDs found in the order")
            # If debugging, try to show the full order_items to diagnose the issue
            if debug and order_items:
                print(f"Full order_items content: {order_items}")
            return None
        
        print(f"Found product IDs: {product_ids}")
        
        # Step 3: Query MongoDB for product info using each product_id
        products_collection = mongo_db['products']
        
        for product_id in product_ids:
            try:
                if debug:
                    print(f"Searching for product_id: {product_id} (type: {type(product_id)})")
                
                # Handle both string and integer product IDs
                if isinstance(product_id, str) and product_id.isdigit():
                    # Try both string and integer versions of the ID
                    product = products_collection.find_one({'product_id': int(product_id)}, {'_id': 0})
                    if not product:
                        product = products_collection.find_one({'product_id': product_id}, {'_id': 0})
                else:
                    product = products_collection.find_one({'product_id': product_id}, {'_id': 0})
                
                if product:
                    print(f"\nProduct information for product_id {product_id}:")
                    print(f"  Name: {product.get('product_name')}")
                    print(f"  Price: {product.get('unit_price')}")
                    print(f"  Brand: {product.get('brand')}")
                    print(f"  Color: {product.get('colour')}")
                    print(f"  Gender: {product.get('gender')}")
                    print(f"  Description: {product.get('description')}")
                    
                    if 'sizes' in product and product['sizes']:
                        print(f"  Available sizes: {', '.join(str(size) for size in product['sizes'])}")
                    
                    if 'reviews' in product and product['reviews']:
                        try:
                            avg_rating = sum(review['rating'] for review in product['reviews']) / len(product['reviews'])
                            print(f"  Average rating: {avg_rating:.1f} ({len(product['reviews'])} reviews)")
                        except (KeyError, TypeError) as e:
                            print(f"  Error calculating average rating: {e}")
                else:
                    print(f"Product with ID {product_id} not found in MongoDB")
                    if debug:
                        # Show a sample document to help diagnose schema issues
                        sample = products_collection.find_one({}, {'_id': 0})
                        if sample:
                            print(f"Sample product document structure: {sample.keys()}")
            except Exception as e:
                print(f"Error retrieving product {product_id}: {e}")
                if debug:
                    traceback.print_exc()
    
    except Exception as e:
        print(f"Error during test: {e}")
        if debug:
            traceback.print_exc()
    
    finally:
        # Close connections
        if 'cursor' in locals():
            cursor.close()
        
        if 'pg_conn' in locals():
            pg_conn.close()
            print("PostgreSQL connection closed")
        
        if 'mongo_client' in locals():
            mongo_client.close()
            print("MongoDB connection closed")

# Run the test
if __name__ == "__main__":
    test_get_order_and_product_info(debug=True)

Running in debug mode - will print detailed information
PostgreSQL connection established
MongoDB collections: ['dummy_collection', 'products']
Found 4 products in MongoDB
MongoDB connection established
Available columns in orders table: ['order_id', 'order_tms', 'customer_id', 'order_status', 'store_id']
Error executing SELECT query: column o.order_items does not exist
LINE 2: ...r_id, o.order_tms, o.order_status, o.customer_id, o.order_it...
                                                             ^
HINT:  Perhaps you meant to reference the column "o.order_tms".

SQL trace:
Error during test: column o.order_items does not exist
LINE 2: ...r_id, o.order_tms, o.order_status, o.customer_id, o.order_it...
                                                             ^
HINT:  Perhaps you meant to reference the column "o.order_tms".

PostgreSQL connection closed
MongoDB connection closed


Traceback (most recent call last):
  File "/tmp/ipykernel_17850/3276926936.py", line 80, in test_get_order_and_product_info
    cursor.execute("""
  File "/home/msis/MDBD/MD/lib/python3.10/site-packages/psycopg2/extras.py", line 236, in execute
    return super().execute(query, vars)
psycopg2.errors.UndefinedColumn: column o.order_items does not exist
LINE 2: ...r_id, o.order_tms, o.order_status, o.customer_id, o.order_it...
                                                             ^
HINT:  Perhaps you meant to reference the column "o.order_tms".

Traceback (most recent call last):
  File "/tmp/ipykernel_17850/3276926936.py", line 80, in test_get_order_and_product_info
    cursor.execute("""
  File "/home/msis/MDBD/MD/lib/python3.10/site-packages/psycopg2/extras.py", line 236, in execute
    return super().execute(query, vars)
psycopg2.errors.UndefinedColumn: column o.order_items does not exist
LINE 2: ...r_id, o.order_tms, o.order_status, o.customer_id, o.order_it...
              